In [1]:
import sys
sys.path.append("/Users/michael/Desktop/orig/qbm-rl-steering/")

In [2]:
import os

import datetime
import shutil
import pickle

import numpy as np
import matplotlib

from tensorflow.keras.optimizers.schedules import PolynomialDecay, PiecewiseConstantDecay
from qbm_rl_steering.core.ddpg_agents import ClassicalDDPG, QuantumDDPG
from qbm_rl_steering.environment.rms_env_nd import RmsSteeringEnv
from qbm_rl_steering.environment.orig_awake_env import e_trajectory_simENV
from qbm_rl_steering.core.run_utils import trainer, evaluator, plot_training_log, plot_evaluation_log

import pandas as pd

! Cannot import libraries required for QAOA ...


In [3]:
# No additional exploration when reloading agents to continue training
# Not sure what to do with the learning rate schedules, action noise and epsilon exploration...
# Maybe start from slightly lower ... a bit strange potentially.
params = {
    'quantum_ddpg': True,
    'n_steps': 440,
    'env/max_steps_per_episode': 8,
    'env/required_steps_above_reward_threshold': 1,
    'trainer/batch_size': 16,
    'trainer/n_exploration_steps': 400,
    'trainer/n_episodes_early_stopping': 20000,
    'agent/gamma': 0.97,
    'agent/tau_critic': 0.00013,
    'agent/tau_actor': 0.0005,
    'lr_critic/init': 0.00025,
    'lr_critic/decay_factor': 1.,
    'lr_actor/init': 0.00032,
    'lr_actor/decay_factor': 1., 
    'lr/final': 5e-5,
    'action_noise/init': 0.3,
    'action_noise/final': 0.,
    'epsilon_greedy/init': 0.5,
    'epsilon_greedy/final': 0., 
    'anneals/n_pieces': 2,
    'anneals/init': 1,
    'anneals/final': 1,  
}

In [4]:
env = e_trajectory_simENV()

# Learning rate schedules
lr_schedule_critic = PolynomialDecay(
    params['lr_critic/init'], params['n_steps'],
    end_learning_rate=params['lr/final'])
lr_schedule_actor = PolynomialDecay(
    params['lr_actor/init'], params['n_steps'],
    end_learning_rate=params['lr/final'])

Exception ...


/Users/michael/Desktop/orig/qbm-rl-steering/qbm_rl_steering/environment/utils/twissReader.py:247: UserWarning: MISSING FIELD PX IN TWISS INPUT /Users/michael/Desktop/qbm-rl-steering/qbm_rl_steering/environment/utils/electron_tt43.out
  warnings.warn('MISSING FIELD ' + FIELD_PX + ' IN TWISS INPUT ' + inputFile)
/Users/michael/Desktop/orig/qbm-rl-steering/qbm_rl_steering/environment/utils/twissReader.py:262: UserWarning: MISSING FIELD PY IN TWISS INPUT/Users/michael/Desktop/qbm-rl-steering/qbm_rl_steering/environment/utils/electron_tt43.out
  warnings.warn('MISSING FIELD ' + FIELD_PY + ' IN TWISS INPUT' + inputFile)
/Users/michael/miniconda3/envs/dwaveenv/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
if params['quantum_ddpg']:
    agentMy = QuantumDDPG(
        state_space=env.observation_space,
        action_space=env.action_space,
        learning_rate_schedule_critic=lr_schedule_critic,
        learning_rate_schedule_actor=lr_schedule_actor,
        grad_clip_actor=np.inf, grad_clip_critic=np.inf,
        gamma=params['agent/gamma'],
        tau_critic=params['agent/tau_critic'],
        tau_actor=params['agent/tau_actor'],
    )
else:
    agentMy = ClassicalDDPG(
        state_space=env.observation_space,
        action_space=env.action_space,
        learning_rate_critic=params['lr_critic/init'],
        learning_rate_actor=params['lr_actor/init'],
        grad_clip_actor=np.inf, grad_clip_critic=np.inf,
        gamma=params['agent/gamma'],
        tau_critic=params['agent/tau_critic'],
        tau_actor=params['agent/tau_actor'],
    )

2022-05-21 16:25:11.464982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


SETTING PROPER QPU AS SAMPLER
QPU Advantage_system6.1 was selected.
 ! Warning: big_gammas are 'virtual'. We don't know the actual values ... 
SETTING PROPER QPU AS SAMPLER
QPU Advantage_system6.1 was selected.
 ! Warning: big_gammas are 'virtual'. We don't know the actual values ... 


In [6]:
# Action noise schedule
action_noise_schedule = PolynomialDecay(
    params['action_noise/init'], params['n_steps'],
    params['action_noise/final'])

# Epsilon greedy schedule
epsilon_greedy_schedule = PolynomialDecay(
    params['epsilon_greedy/init'], params['n_steps'],
    params['epsilon_greedy/final'])

# Schedule n_anneals
t_transition = [int(x * params['n_steps']) for x in
                np.linspace(0, 1., params['anneals/n_pieces'] + 1)][1:-1]
y_transition = [int(n) for n in np.linspace(params['anneals/init'],
                                            params['anneals/final'],
                                            params['anneals/n_pieces'])]
n_anneals_schedule = PiecewiseConstantDecay(t_transition, y_transition)

# PREPARE OUTPUT FOLDERs
date_time_now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
out_path = './runs/indiv/' + date_time_now
os.makedirs(out_path)

In [7]:
with open(out_path + '/params_dict.pkl', 'wb') as fid:
    pickle.dump(params, fid)

# AGENT TRAINING
episode_log = trainer(
    env=env, agent=agentMy, action_noise_schedule=action_noise_schedule,
    epsilon_greedy_schedule=epsilon_greedy_schedule,
    n_anneals_schedule=n_anneals_schedule, n_steps=params['n_steps'],
    max_steps_per_episode=params['env/max_steps_per_episode'],
    batch_size=params['trainer/batch_size'],
    n_exploration_steps=params['trainer/n_exploration_steps'],
    n_episodes_early_stopping=params['trainer/n_episodes_early_stopping'],
    out_path=out_path)


EPISODE: 0
INITIAL REWARD: -162.804
FINAL REWARD: -394.858
#STEPS: 8 (8 RANDOM)
EARLY STOPPING COUNT: 0/20000

MOVING AVG FINAL REWARD: -394.858
MOVING AVG #STEPS: 8.0

EPISODE: 1
INITIAL REWARD: -245.548
FINAL REWARD: -511.288
#STEPS: 8 (8 RANDOM)
EARLY STOPPING COUNT: 0/20000

MOVING AVG FINAL REWARD: -453.073
MOVING AVG #STEPS: 8.0

EPISODE: 2
INITIAL REWARD: -286.527
FINAL REWARD: -524.608
#STEPS: 8 (8 RANDOM)
EARLY STOPPING COUNT: 0/20000

MOVING AVG FINAL REWARD: -476.918
MOVING AVG #STEPS: 8.0

EPISODE: 3
INITIAL REWARD: -137.536
FINAL REWARD: -33.357
#STEPS: 1 (1 RANDOM)
EARLY STOPPING COUNT: 1/20000

MOVING AVG FINAL REWARD: -366.028
MOVING AVG #STEPS: 6.2

EPISODE: 4
INITIAL REWARD: -224.418
FINAL REWARD: -77.625
#STEPS: 3 (3 RANDOM)
EARLY STOPPING COUNT: 2/20000

MOVING AVG FINAL REWARD: -308.347
MOVING AVG #STEPS: 5.6

EPISODE: 5
INITIAL REWARD: -258.905
FINAL REWARD: -1479.27
#STEPS: 8 (8 RANDOM)
EARLY STOPPING COUNT: 0/20000

MOVING AVG FINAL REWARD: -503.501
MOVING AVG 

1/1 [==============================] - 0s 11ms/step

EPISODE: 84
INITIAL REWARD: -131.46
FINAL REWARD: -417.547
#STEPS: 8 (5 RANDOM)
EARLY STOPPING COUNT: 0/20000

MOVING AVG FINAL REWARD: -248.618
MOVING AVG #STEPS: 4.5


Learning progress -- Episode 85:   0%|                                                  | 0/8 [00:00<?, ?it/s]

Calling main_critic to calculate q values on batch in get_analytical_action_derivative ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16
Looping over batch of samples, getting current and future Q values in _update_critic ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16


/Users/michael/Desktop/orig/qbm-rl-steering/qbm_rl_steering/core/ddpg_agents.py:496: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target_weights = np.array(self.target_actor_net.get_weights())
/Users/michael/Desktop/orig/qbm-rl-steering/qbm_rl_steering/core/ddpg_agents.py:497: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  main_weights = np.array(self.main_actor_net.get_weights())
Learning progress -- Episode 85:  12%|████▊                                 | 1/8 [34:51<4:03:59, 2091.39s/it]

...

Calling main_critic to calculate q values on batch in get_analytical_action_derivative ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16
Looping over batch of samples, getting current and future Q values in _update_critic ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16


Learning progress -- Episode 98:  50%|████████████████████                    | 1/2 [27:43<27:43, 1663.91s/it]

SAVING EVERYTHING: Ep 98, stp 0
Calling main_critic to calculate q values on batch in get_analytical_action_derivative ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16
Looping over batch of samples, getting current and future Q values in _update_critic ...
Sample in batch: 1/16
Sample in batch: 5/16
Sample in batch: 9/16
Sample in batch: 13/16


Learning progress -- Episode 98: 100%|████████████████████████████████████████| 2/2 [57:54<00:00, 1737.00s/it]

SAVING EVERYTHING: Ep 98, stp 1
n_total_steps_training 441


In [8]:
plot_training_log(env, agentMy, episode_log, apply_scaling=True,
                  save_path=out_path)

In [9]:
# n_training_episodes = len(episode_log['final_rewards'])
episode_log_2 = {}
for k, v in episode_log.items():
    if len(v) != 0:
        episode_log_2[k] = v[:]
episode_log = episode_log_2

df_train_log = pd.DataFrame(episode_log)
df_train_log.to_csv(out_path + '/train_log')

# Save agent
weights = {
    'main_critic': {
        'w_vh': agentMy.main_critic_net.w_vh,
        'w_hh': agentMy.main_critic_net.w_hh},
    'target_critic': {
        'w_vh': agentMy.target_critic_net.w_vh,
        'w_hh': agentMy.target_critic_net.w_hh}}

with open(out_path + '/critic_weights.pkl', 'wb') as fid:
    pickle.dump(weights, fid)

weights = {'main_actor': agentMy.main_actor_net.get_weights(),
           'target_actor': agentMy.target_actor_net.get_weights()}
with open(out_path + '/actor_weights.pkl', 'wb') as fid:
    pickle.dump(weights, fid)

In [30]:
env = e_trajectory_simENV()
eval_log_scan = evaluator(env, agentMy, n_episodes=50, reward_scan=False)
np.save(out_path + '/eval_log_scan.npy', eval_log_scan)

Exception ...
1/1 [==============================] - 0s 12ms/step


In [31]:
plot_evaluation_log(
    env, env.MAX_TIME+1, eval_log_scan, type='random', apply_scaling=True,
    save_path=out_path)